In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

# model imports
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import train_test_split

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier as KNN

import xgboost as xgb

# imports from starter code by SRK
import os
import sys
import operator
from scipy import sparse
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [13]:
data_path = "../data/"
train_file = data_path + "churn_test.csv"
test_file = data_path + "churn_train.csv"
train_df = pd.read_csv(train_file, parse_dates=['last_trip_date', 'signup_date'], infer_datetime_format=True)
test_df = pd.read_csv(test_file, parse_dates=['last_trip_date', 'signup_date'], infer_datetime_format=True)
print(train_df.shape)
print(test_df.shape)

(10000, 12)
(40000, 12)


In [16]:
train_df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,2.48,5.0,5.0,1.0,Winterfell,2014-01-07,Android,2014-01-06,0.0,2,True,100.0
1,10.81,5.0,5.0,1.0,Winterfell,2014-04-29,iPhone,2014-01-06,0.0,3,True,100.0
2,12.95,5.0,5.0,1.0,Astapor,2014-01-29,Android,2014-01-19,0.0,1,True,100.0
3,3.92,5.0,NaN,1.0,Winterfell,2014-02-16,iPhone,2014-01-09,0.0,0,False,0.0
4,1.46,5.0,4.5,1.0,Astapor,2014-01-09,iPhone,2014-01-07,0.0,2,False,100.0


In [26]:
has_nulls = []
for column in train_df.columns:
    if train_df[column].isnull().sum() > 0:
        has_nulls.append(column)
        print("column {} has {} nulls".format(column, train_df[column].isnull().sum()))
        print(train_df[column].describe())
        print("")

column avg_rating_by_driver has 39 nulls
count    9961.000000
mean        4.781056
std         0.440871
min         1.000000
25%              NaN
50%              NaN
75%              NaN
max         5.000000
Name: avg_rating_by_driver, dtype: float64

column avg_rating_of_driver has 1594 nulls
count    8406.000000
mean        4.601011
std         0.627343
min         1.000000
25%              NaN
50%              NaN
75%              NaN
max         5.000000
Name: avg_rating_of_driver, dtype: float64

column phone has 77 nulls
count       9923
unique         2
top       iPhone
freq        6954
Name: phone, dtype: object



/opt/conda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [27]:
has_nulls = []
for column in test_df.columns:
    if test_df[column].isnull().sum() > 0:
        has_nulls.append(column)
        print("column {} has {} nulls".format(column, test_df[column].isnull().sum()))
        print(test_df[column].describe())
        print("")
has_nulls

column avg_rating_by_driver has 162 nulls
count    39838.000000
mean         4.777434
std          0.448088
min          1.000000
25%               NaN
50%               NaN
75%               NaN
max          5.000000
Name: avg_rating_by_driver, dtype: float64

column avg_rating_of_driver has 6528 nulls
count    33472.000000
mean         4.601697
std          0.614810
min          1.000000
25%               NaN
50%               NaN
75%               NaN
max          5.000000
Name: avg_rating_of_driver, dtype: float64

column phone has 319 nulls
count      39681
unique         2
top       iPhone
freq       27628
Name: phone, dtype: object



/opt/conda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


['avg_rating_by_driver', 'avg_rating_of_driver', 'phone']

In [ ]:
# Replacing ratings with means

for column in ['avg_rating_by_driver', 'avg_rating_of_driver']:
    train_df[train_df[column].isnull()] = train_df[column].mean()
    test_df[test_df[column].isnull()] = test_df[column].mean()

train_df['phone'] = 

4.781056118863625